<a href="https://colab.research.google.com/github/serna01/GNSS/blob/main/homework2_GNSS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#runway 30 using WGS 84 XYZ coordinates -ECEF
import numpy as np
rwy30start = np.array([-2694684.473,
              -4293642.366,
              3857878.924])

rwy30end = np.array([-2694892.460,
            -4293083.225,
            3858353.437])
print (rwy30start)
print (rwy30end)

[-2694684.473 -4293642.366  3857878.924]
[-2694892.46  -4293083.225  3858353.437]


In [ ]:
#geodetic latitude, longitude, and altitude above the WGS-84 ellipsoid
#of the rwy30start position.Provide the latitude and longitude in degrees, and the
#altitude in meters. Make sure you include enough significant digits to capture the
#accuracy of the XYZ coordinates.

#WGS-84 ellipsoidal parameters
semiMajor_axis = 6378137.0 #meters
flattening_factor = 1/298.257223563 #unitless
semiMinor_axis = semiMajor_axis * (1 - flattening_factor)
we = 7292115.0e-10 #rad/s
G = 3986004.418e8 #m^3/s^2
c = 299792458.0 #m/s


In [ ]:
import numpy as np

# WGS-84 ellipsoidal parameters
a = 6378137.0  # semi-major axis in meters
f = 1 / 298.257223563  # flattening
b = a * (1 - f)  # semi-minor axis
e_sq = 2*f - f**2  # eccentricity squared

def iterative_ecef_to_geodetic(x, y, z, tolerance=1e-10, max_iterations=100):
    """
    Convert ECEF coordinates to geodetic coordinates using an iterative method.

    Args:
    x, y, z (float): ECEF coordinates in meters
    tolerance (float): Convergence tolerance
    max_iterations (int): Maximum number of iterations

    Returns:
    tuple: (latitude, longitude, altitude) where lat and lon are in degrees, alt in meters
    """
    # Calculate longitude
    lon = np.arctan2(y, x)

    # Initial guess for latitude
    p = np.sqrt(x**2 + y**2)
    lat = np.arctan2(z, p * (1 - e_sq))

    # Iterative calculation of latitude and altitude
    for _ in range(max_iterations):
        N = a / np.sqrt(1 - e_sq * np.sin(lat)**2)
        h = p / np.cos(lat) - N

        lat_new = np.arctan2(z, p * (1 - e_sq * N / (N + h)))

        if np.abs(lat - lat_new) < tolerance:
            break

        lat = lat_new

    return np.degrees(lat), np.degrees(lon), h

# Example usage:

print ("Runway 30 Start")
lat, lon, alt = iterative_ecef_to_geodetic(*rwy30start)

print(f"Latitude:  {lat:.9f} degrees")
print(f"Longitude: {lon:.9f} degrees")
print(f"Altitude:  {alt:.4f} meters")

print ("Runway 30 End")
lat_end, lon_end, alt_end = iterative_ecef_to_geodetic(*rwy30end)

print(f"Latitude:  {lat_end:.9f} degrees")
print(f"Longitude: {lon_end:.9f} degrees")
print(f"Altitude:  {alt_end:.4f} meters")

Runway 30 Start
Latitude:  37.458379348 degrees
Longitude: -122.112329422 degrees
Altitude:  -31.8775 meters
Runway 30 End
Latitude:  37.463762427 degrees
Longitude: -122.117680472 degrees
Altitude:  -31.4155 meters


In [ ]:
#Determine the latitude and longitude of the rwy30Start and rwy30End position in
#degrees, arc-minutes, and arc-seconds.

#define my other functions

def to_arc_mins(degrees: float) -> float:
  return degrees * 60

def to_arc_secs(degrees: float) -> float:
  return degrees * 3600

print ("Runway 30 Start")
print(f"Latitude in arc mins:  {to_arc_mins(lat):.9f} degrees")
print(f"Longitude in arc mins:  {to_arc_mins(lon):.9f} degrees\n")
print(f"Latitude in arc secs:  {to_arc_secs(lat):.9f} degrees")
print(f"Longitude in arc sec:  {to_arc_secs(lon):.9f} degrees \n")

print ("Runway 30 End")
print(f"Latitude in arc mins:  {to_arc_mins(lat_end):.9f} degrees")
print(f"Longitude in arc mins:  {to_arc_mins(lon_end):.9f} degrees\n")
print(f"Latitude in arc secs:  {to_arc_secs(lat_end):.9f} degrees")
print(f"Longitude in arc secs:  {to_arc_secs(lon_end):.9f} degrees\n")



Runway 30 Start
Latitude in arc mins:  2247.502760893 degrees
Longitude in arc mins:  -7326.739765341 degrees

Latitude in arc secs:  134850.165653601 degrees
Longitude in arc sec:  -439604.385920487 degrees 

Runway 30 End
Latitude in arc mins:  2247.825745604 degrees
Longitude in arc mins:  -7327.060828342 degrees

Latitude in arc secs:  134869.544736221 degrees
Longitude in arc secs:  -439623.649700543 degrees



In [ ]:
#Given that in the vicinity of Palo Alto the geoid is 33 meters below the WGS 84
#reference ellipsoid, what is the height of the rwy30Start position above the geoid?
alt_geoid_paloAlto = 33 #meters
print(f"height of the rwy30Start position above the geoid in Palo Alto is {(alt_geoid_paloAlto + alt):.5} meters")



height of the rwy30Start position above the geoid in Palo Alto is 1.1225 meters


In [ ]:
def ecef_to_enu(origin, point):
    """
    Convert ECEF coordinates to ENU coordinates.
    using info from https://gssc.esa.int/navipedia/index.php/Transformations_between_ECEF_and_ENU_coordinates

    Parameters:
    origin (numpy array): ECEF coordinates of the origin (reference frame) [X, Y, Z].
    point (numpy array): ECEF coordinates of the point to convert [X, Y, Z].

    Returns:
    numpy array: ENU coordinates of the point relative to the origin.
    """
    # Extract the X, Y, Z coordinates of the origin
    Xo, Yo, Zo = origin

    # lat and long come from previous calculations

    latitude = lat
    longitude = lon

    # Compute the transformation matrix R
    sin_lat = np.sin(latitude)
    cos_lat = np.cos(latitude)
    sin_lon = np.sin(longitude)
    cos_lon = np.cos(longitude)

    R = np.array([
        [-sin_lon, cos_lon, 0],
        [-sin_lat * cos_lon, -sin_lat * sin_lon, cos_lat],
        [cos_lat * cos_lon, cos_lat * sin_lon, sin_lat]
    ])

    # Calculate the position vector from the origin to the point
    position_vector = point - origin

    # Convert the position vector to ENU coordinates using the transformation matrix
    enu_coordinates = R @ position_vector

    return enu_coordinates, latitude, longitude


In [ ]:
enu_coordinates, lati, longi = ecef_to_enu(rwy30start, rwy30end)
print("ENU Coordinates:", enu_coordinates)

ENU Coordinates: [-595.71233622  453.20026963 -144.20875596]


In [ ]:
def local_to_utc(year, month, day, hour, minute, second, timezone_offset):
    """
    Convert local time to UTC.

    Parameters:
    year (int): Year
    month (int): Month
    day (int): Day
    hour (int): Hour
    minute (int): Minute
    second (int): Second
    timezone_offset (int): Timezone offset in hours

    Returns:
    tuple: UTC time as (year, month, day, hour, minute, second)
    """
    # Adjust time for timezone offset
    hour -= timezone_offset

    # Handle overflow or underflow of hours
    if hour < 0:
        hour += 24
        day -= 1
    elif hour >= 24:
        hour -= 24
        day += 1

    # Simple month and day handling, not accounting for month length or leap years
    if month == 2 and day > 28:
        day -= 28
        month += 1
    elif day > 30:  # Handle months with 30 days
        day -= 30
        month += 1
    if month > 12:  # Handle year overflow
        month = 1
        year += 1

    return year, month, day, hour, minute, second


def local_to_utc_jd(year, month, day, hour, minute, second):
    """
    Convert local time to Julian Date and Modified Julian Date.

    Parameters:
    year (int): Year
    month (int): Month
    day (int): Day
    hour (int): Hour
    minute (int): Minute
    second (int): Second

    Returns:
    tuple: Julian Date (JD) and Modified Julian Date (MJD)
    """
    if month <= 2:
        year -= 1
        month += 12
    A = year // 100
    B = 2 - A + (A // 4)
    JD = int(365.25 * (year + 4716)) + int(30.6001 * (month + 1)) + day + B - 1524.5 + (hour + minute / 60 + second / 3600) / 24
    MJD = JD - 2400000.5
    return JD, MJD


def local_to_gpst(year, month, day, hour, minute, second):
    """
    Convert local time to GPS Time (week and seconds of week).

    Parameters:
    year (int): Year
    month (int): Month
    day (int): Day
    hour (int): Hour
    minute (int): Minute
    second (int): Second

    Returns:
    tuple: GPS Week and seconds of the week
    """
    # Convert local time to UTC
    utc_year, utc_month, utc_day, utc_hour, utc_minute, utc_second = local_to_utc(year, month, day, hour, minute, second, 0)

    # Calculate Julian Date
    JD, _ = local_to_utc_jd(utc_year, utc_month, utc_day, utc_hour, utc_minute, utc_second)

    # GPS epoch starts on January 6, 1980
    gps_epoch_jd = 2444244.5  # JD for January 6, 1980
    gps_time_days = JD - gps_epoch_jd
    gps_week = int(gps_time_days // 7)
    gps_seconds_of_week = gps_time_days * 86400 % 604800  # Seconds in a week

    return gps_week, gps_seconds_of_week


def local_to_tai(year, month, day, hour, minute, second, timezone_offset):
    """
    Convert local time to TAI.

    Parameters:
    year (int): Year
    month (int): Month
    day (int): Day
    hour (int): Hour
    minute (int): Minute
    second (int): Second
    timezone_offset (int): Timezone offset in hours

    Returns:
    tuple: TAI as seconds since epoch (1970-01-01 00:00:00 UTC) and total leap seconds added
    """
    # Leap seconds table (as of 2024, total leap seconds = 38)
    leap_seconds_table = {
        1958: 0, 1960: 1, 1961: 2, 1962: 3, 1963: 4, 1964: 5,
        1965: 6, 1966: 7, 1967: 8, 1968: 9, 1969: 10, 1970: 11,
        1971: 12, 1972: 13, 1973: 14, 1974: 15, 1975: 16, 1976: 17,
        1977: 18, 1978: 19, 1979: 20, 1981: 21, 1982: 22, 1983: 23,
        1985: 24, 1987: 25, 1989: 26, 1990: 27, 1992: 28, 1993: 29,
        1994: 30, 1995: 31, 1997: 32, 1998: 33, 2005: 34, 2008: 35,
        2012: 36, 2015: 37, 2016: 38  # Last recorded leap second
    }

    # Convert local time to UTC
    utc_year, utc_month, utc_day, utc_hour, utc_minute, utc_second = local_to_utc(
        year, month, day, hour, minute, second, timezone_offset)

    # Calculate total leap seconds based on UTC year
    leap_seconds = 0
    for year_key in leap_seconds_table:
        if year_key < utc_year or (year_key == utc_year and (utc_month > 1 or (utc_month == 1 and utc_day >= 1))):
            leap_seconds = leap_seconds_table[year_key]

    # Calculate the UTC timestamp in seconds since the epoch
    utc_seconds = (
        (utc_year - 1970) * 365 * 24 * 3600 +   # Years
        ((utc_year - 1970) // 4) * 24 * 3600 +  # Leap years
        (utc_month - 1) * 30 * 24 * 3600 +      # Months (simplified)
        (utc_day - 1) * 24 * 3600 +              # Days
        utc_hour * 3600 + utc_minute * 60 + utc_second  # Time
    )

    # Add leap seconds to convert UTC to TAI
    tai_seconds = utc_seconds + leap_seconds

    return tai_seconds, leap_seconds


# Example usage
year, month, day, hour, minute, second = 2023, 10, 19, 15, 30, 0
timezone_offset = -5  # For example, UTC-5

jd, mjd = local_to_utc_jd(year, month, day, hour, minute, second)
gps_week, gps_seconds = local_to_gpst(year, month, day, hour, minute, second)
tai_seconds, tai_leap_seconds = local_to_tai(year, month, day, hour, minute, second, timezone_offset)

print(f"Julian Date: {jd}, Modified Julian Date: {mjd}")
print(f"GPS Week: {gps_week}, Seconds of Week: {gps_seconds}")
print(f"TAI (in seconds since epoch): {tai_seconds}, Leap Seconds: {tai_leap_seconds}")


Julian Date: 2460237.1458333335, Modified Julian Date: 60236.64583333349
GPS Week: 2284, Seconds of Week: 401400.00001335144
TAI (in seconds since epoch): 1697488238, Leap Seconds: 38
